In [3]:
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
from rouge import Rouge

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-ru-en.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-ru.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


Translated Summary (Russian): История театра началась 17 марта (28) 1776 года, когда прокуратура принц Питер Василиевич Урусов получила наивысшее одобрение Императрицы Екатерины II.
Rouge Score: [{'rouge-1': {'r': 0.02857142857142857, 'p': 0.08695652173913043, 'f': 0.04301074896519861}, 'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0}, 'rouge-l': {'r': 0.02857142857142857, 'p': 0.08695652173913043, 'f': 0.04301074896519861}}]


In [ ]:
# Загрузка предварительно обученных моделей и токенизаторов для суммаризации и перевода
summarization_model_name = "Helsinki-NLP/opus-mt-ru-en"  # Мультиязычная модель для суммаризации
translation_model_name = "Helsinki-NLP/opus-mt-en-ru"     # Модель перевода с английского на русский

In [ ]:
summarization_tokenizer = AutoTokenizer.from_pretrained(summarization_model_name)
summarization_model = TFAutoModelForSeq2SeqLM.from_pretrained(summarization_model_name)

translation_tokenizer = AutoTokenizer.from_pretrained(translation_model_name)
translation_model = TFAutoModelForSeq2SeqLM.from_pretrained(translation_model_name)

In [ ]:
# Функция для суммаризации и перевода текста
def summarize_and_translate(text):
    # Суммаризация текста на английском
    inputs = summarization_tokenizer(text, return_tensors="tf", max_length=1024, truncation=True)
    summary_ids = summarization_model.generate(inputs["input_ids"], max_length=50, min_length=10, num_beams=4, early_stopping=True)
    summary_en = summarization_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    # Перевод суммаризированного текста на русский
    translation_inputs = translation_tokenizer(summary_en, return_tensors="tf", max_length=1024, truncation=True)
    translation_ids = translation_model.generate(translation_inputs["input_ids"], max_length=1024, num_beams=4, early_stopping=True)
    translated_summary_ru = translation_tokenizer.decode(translation_ids[0], skip_special_tokens=True)
    
    return summary_en, translated_summary_ru

In [ ]:
# Пример текста на русском для суммаризации и перевода
example_text_ru = """
Историю театра принято вести с 17 (28) марта 1776 года, когда губернский прокурор князь Пётр Васильевич Урусов получил высочайшее соизволение императрицы Екатерины II «содержать… театральные всякого рода представления, а также концерты, воксалы и маскарады». Князь начал строительство театра, который — по месту расположения на улице Петровка (на правом берегу Неглинки) — был назван Петровским[2]. Театр Урусова сгорел ещё до его открытия, и князь передал дела своему компаньону, английскому предпринимателю Майклу (Михаилу) Меддоксу. 
"""

# Суммаризация и перевод текста на русский
summary_en, translated_summary_ru = summarize_and_translate(example_text_ru)

In [ ]:
# Расчет метрики Rouge
rouge = Rouge()
rouge_score = rouge.get_scores(summary_en, example_text_ru)

In [ ]:
# Вывод переведенного текста и метрики Rouge
print("Переведенная суммаризация (на русском):", translated_summary_ru)
print("Оценка Rouge:", rouge_score)